# LATEST VERSION

In [12]:
import time
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

In [13]:
item_info_df = pd.read_csv("item_info.csv").drop(columns=["min_age", "max_age"])
item_info_df["item_no"] = item_info_df["item_no"].astype(str)
user_info_df = pd.read_csv("user_info.csv")
user_info_df["user_no"] = user_info_df["user_no"].astype(str)
user_item_interaction_df = pd.read_csv("user_item_interactions.csv")
user_item_interaction_df["user_no"] = user_item_interaction_df["user_no"].astype(str)
user_item_interaction_df["item_no"] = user_item_interaction_df["item_no"].astype(str)

In [14]:
item_info_df.head()

,item_no,colour,gender_description,brand,product_group
0,206890150141030846,beige,unisex,bloomingville,furniture
1,7637494654837559066,pink,girls,petit bateau,clothing sets
2,7969520735315050609,beige,unisex,bobo choses,jumpers and knitwear
3,-565751122846696741,white,unisex,piupiuchick,tops
4,6688930722259797984,green,unisex,filibabba,baby feeding


In [15]:
user_info_df.head()

,user_no,country,aov,avg_markdown
0,8587933979694308845,sweden,479.2,-0.200000
1,2740387653650048572,sweden,837.6,-0.200000
2,2084988796719274722,sweden,942.4,-0.560000
3,-4662401680846085311,sweden,438.4,-0.620000
4,-1086148156436227367,sweden,664.8,-0.286667


In [16]:
user_item_interaction_df.head()

,user_no,item_no,date,eventtype
0,9060639138425951676,-478270421339298398,2021-06-24,purchased
1,9060639138425951676,2658388892627023500,2021-11-27,pageView
2,9060639138425951676,504233002877562247,2021-11-26,pageView
3,9060639138425951676,-3413566329152665076,2021-10-21,pageView
4,9060639138425951676,6700254580945881296,2021-10-10,pageView


In [17]:
user_item_interaction_df["eventtype"].value_counts()

pageView         411685
addToCart        354022
purchased        227165
addToWishlist     18353
Name: eventtype, dtype: int64

In [ ]:
USER_FEATURES = [
    "aov"
]
ITEM_FEATURES = [
    "brand"
]

# Notebook structure
Overall thought: Make a simple example using one or two features, but providing a working example to inspire the audience to go test by themselves, since there are more features in the data. Same logic with the model complexity.

Use a lot of the stuff from the TFRS tutorials when it comes to batch-sizes and loss functions to make it "relatable".

## Load data
* Whats done above this cell already
## Prepare data
* Choose features
* Preprocess (missing values / standardize  etc)
* Split data train/test
* Create TF Dataset
## Model creation
* Create one Query Model and one Candidate model (use same terminology as in the presentation). Maybe skip the dense layers?
* Create one TFRS model, with appropriate task and loss function using the two tower model architecture.
## Model training
* Would be nice if we had it set up so you could train both on GPU or CPU. So baiscally a flag if you want to use GPU or not (or check if GPU attached automatically). 
* Train for n epochs
* Important to set training=False in the loss function, otherwise it evaluates during every epoch on the train data, which is painfully slow. So something like (def compute_loss(self, features, training=False) -> tf.Tensor:)

## Model evaluation
* Evaluate on test and train, show difference between Top-100 accuracy
* Show that it overfits easily, just like any NN, so need to apply regularization and early stopping if you want it to generalize well (maybe not implement this)

## Model serving
* Create an index, and predict on one customer, show speed of inference. Batch-predict over all users and show ms/user of inference speed.